In [1]:
import ee
import geemap
import math 

In [2]:
Map = geemap.Map()

In [2]:
print('Hello world')

Hello world


In [3]:
ee.Initialize()

In [4]:
from utils.utils import *
from utils.ee_utils import *
from utils.utils import TextColors as c

In [5]:
# df = read_csv("LUCAS_2015_all.csv")
df = pd.read_csv('RapidCarbon.csv')
df

,Point_ID,lat,long,SOC
0,A0108P92,48.85,-122.68,100.113941
1,A0116P90,47.24,-122.08,242.002573
2,A0116P94,47.24,-122.09,293.079581
3,A0116X97,42.49,-113.18,69.735244
4,A0120R93,45.23,-117.07,116.996856
...,...,...,...,...
5935,F1809F96,35.99,-81.82,104.674321
5936,F1809F97,35.36,-83.92,256.694425
5937,F1809F98,34.73,-83.84,235.398204
5938,F1809F99,35.34,-83.95,106.539187


In [7]:
DATASET = 'RaCA' #'LUCAS'

if DATASET == 'RaCA':
 create_folder_if_not_exists('l8_images_us')
 DFOLDER = 'l8_images_us//'

if DATASET == 'LUCAS':
 create_folder_if_not_exists('l8_images')
 DFOLDER = 'l8_images//'

Folder "l8_images_us" already exists in the current working directory.


### Ignore the following cell, some kind of data cleaning

In [9]:
# # Step 1: Get the list of TIFF file names
# tif_folder_path = "C:\\Users\\nkakhani\\_Multimodal\\SoilNet-7\\SoilNet-PreRelease\\dataset\\l8_images_us"
# tif_files = [f for f in os.listdir(tif_folder_path) if f.endswith('.tif')]

# # Extract the first part of each TIFF file name
# tiff_first_parts = [tif.split('_')[0] for tif in tif_files]
# print(len(tiff_first_parts))

# # Step 3: Find Point IDs in the CSV DataFrame that are not in tiff_first_parts
# missing_point_ids = df[~df['Point_ID'].isin(tiff_first_parts)]['Point_ID']

# # Print or use missing_point_ids as needed
# print("Point IDs in CSV DataFrame not present in TIFF file names:")
# print(missing_point_ids)

# # # Step 3: Check if each first part is present in the specific column of the CSV file
# # for tiff_first_part, tif_file in zip(tiff_first_parts, tif_files):
# #     if tiff_first_part not in df['Point_ID'].values:
# #         # Step 4: If not present, delete the file immediately
# #         file_path = os.path.join(tif_folder_path, tif_file)
# #         os.remove(file_path)
# #         print(f"Deleted {tif_file}")

In [ ]:
# Config
IMAGE_COLLECTION = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
WINTER_RANGES = [('2015-01-01', '2015-02-28'), ('2015-11-01', '2015-12-31')]
NONE_WINTER_RANGES = (WINTER_RANGES[0][1], WINTER_RANGES[1][0])
#Required Landsat bands and RS indices
BANDS = ['SR_B1','SR_B2','SR_B3','SR_B4','SR_B5','SR_B6','SR_B7','clayIndex',
         'ferrousIndex','carbonateIndex','rockOutcropIndex','NDVI','elevation','slopePCT']

In [ ]:
winter_ragne_1 = ee.Filter.date(WINTER_RANGES[0][0], WINTER_RANGES[0][1])
winter_range_2 = ee.Filter.date(WINTER_RANGES[1][0], WINTER_RANGES[1][1])
winter_range_filter = ee.Filter.Or(winter_ragne_1, winter_range_2)

non_winter_range_filter = ee.Filter.date(NONE_WINTER_RANGES[0], NONE_WINTER_RANGES[1])

for index, row in df.iterrows():
    # print(f'{c.BOLD} Point {int(row["Piont_id"])} {c.ENDC}' , end=" Cheking for Winter Image -> ")
    print(f'{c.BOLD} Point {(row["Piont_id"])} {c.ENDC}' , end=" Cheking for Winter Image -> ")
    loop_roi = get_square_roi(row['lat'],row['long'], roi_size=1920, return_gee_object=True)
    
    # FIRST: Checkig if we can find an image in the WINTER_RANGE
    l8 = IMAGE_COLLECTION.filter(winter_range_filter)\
                         .filterBounds(loop_roi).sort('system:time_start')
    if l8.size().getInfo() > 0:                     
        #clip the image collection to the roi                     
        l8 = l8.map(lambda img: img.clip(loop_roi))
        topoBands = add_topo().clip(loop_roi)
        
        # discard images with high null pixels
        l8 = l8.map(lambda img: img.set('not_null_pixels', get_not_nulls_ratio(img,loop_roi)))
        l8 = l8.filter(ee.Filter.gt('not_null_pixels',0.7))
        
        # Ratiometric correction
        l8 = l8.map(lambda img: radiometric_correction(img))
        # add roi cloud cover and cloud shadow property # the index 2 is for the combination of cloud and cloud shadow mask
        l8 = l8.map(lambda img: img.set('roi_cloud_cover', get_mask_ones_ratio(get_cloud_mask(img)[2])))
        l8 = l8.filter(ee.Filter.lt('roi_cloud_cover',10)) 
        
        # add roi snow cover property
        l8 = l8.map(lambda img: img.set('roi_snow_cover', get_mask_ones_ratio(get_snow_mask(img))))
        l8 = l8.filter(ee.Filter.lt('roi_snow_cover',10)) 

    if l8.size().getInfo() > 0:
        print (f'{c.OKGREEN} Found {c.ENDC}')
        l8_image = add_mineral_indices(l8.sort('system:time_start').first()).addBands(topoBands)
        # print('Band names:', l8_image.bandNames().getInfo())

    else: # if we can't find an image in the WINTER_RANGE, we check the NONE_WINTER_RANGE
        print (f'{c.FAIL} Not Found {c.ENDC}', end=" Cheking for None Winter Image -> ")
        l8 = IMAGE_COLLECTION.filter(non_winter_range_filter)\
                             .filterBounds(loop_roi).sort('system:time_start')
                             
        #clip the image collection to the roi                     
        l8 = l8.map(lambda img: img.clip(loop_roi))
                    
        # discard images with high null pixels
        l8 = l8.map(lambda img: img.set('not_null_pixels', get_not_nulls_ratio(img,loop_roi)))
        l8 = l8.filter(ee.Filter.gt('not_null_pixels',0.7))

        # Ratiometric correction
        l8 = l8.map(lambda img: radiometric_correction(img))
        
        # add roi cloud cover and cloud shadow property # the index 2 is for the combination of cloud and cloud shadow mask
        l8 = l8.map(lambda img: img.set('roi_cloud_cover', get_mask_ones_ratio(get_cloud_mask(img)[2])))
        l8 = l8.filter(ee.Filter.lt('roi_cloud_cover',10)) 
        
        # add roi snow cover property
        l8 = l8.map(lambda img: img.set('roi_snow_cover', get_mask_ones_ratio(get_snow_mask(img))))
        l8 = l8.filter(ee.Filter.lt('roi_snow_cover',10)) 
        
        l8 = l8.map(lambda img: img.set('roi_mean_ndvi', get_mean_ndvi(img)))  
        
        if l8.size().getInfo() > 0:
            print(f'{c.OKGREEN} Found {c.ENDC}')
            l8_image = add_mineral_indices(l8.sort('roi_mean_ndvi').first()).addBands(topoBands)
            # print('Band names:', l8_image.bandNames().getInfo())
        else:
            print(f'{c.FAIL} No Image Found For this ROI! {c.ENDC}')

    if l8.size().getInfo() > 0:    
        date = ee.Date(l8_image.get('system:time_start')).format('YYMMdd').getInfo()
        # name = f"{int(row['Piont_id'])}_{date}.tif"
        name = f"{(row['Piont_id'])}_{date}.tif"
        geemap.download_ee_image(l8_image.select(BANDS), DFOLDER+ name,crs='EPSG:3857', scale=30, region = loop_roi)
                       
    #print('Date:',milsec2date(l8.aggregate_array('system:time_start').getInfo()))
    
    # if index == 20:
    #     break

In [ ]:
sdf = (123,)
len(sdf)

## Testing some outputs

In [ ]:
date = ee.Date(l8_image.get('system:time_start')).format('YYMMdd').getInfo()
date

In [ ]:
Map = geemap.Map()
# l8_c_img = l8.sort('roi_mean_ndvi',True).first()
l8_c_img = l8.sort('system:time_start',True).first()
print(l8_c_img.get('roi_mean_ndvi').getInfo())
visualization = {
  'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
  'min': 0.0,
  'max': 0.3,
}

cloud = get_cloud_mask(l8_c_img)[2]


Map.addLayer(cloud, {'min': 0, 'max': 1, 'palette': ['black','red']}, 'cloud')
Map.addLayer(l8_c_img, visualization, 'rgb')
Map.centerObject(loop_roi)
Map